In [58]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [59]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [60]:
df = pd.read_csv(r"C:\MY FILE\MBAN 6110S-Data Science\module_5_feature_engineering_demo.csv")

In [61]:
df.head()


,Unnamed: 0,Height,Weight,Calories_Intake,Exercise_Time,BMI,Age,Resting_Heart_Rate
0,0,187.640523,81.119254,1733.539473,4.593274,23.039374,70,54.540176
1,1,174.001572,87.849478,1644.014918,3.568722,29.015686,38,63.510746
2,2,179.787380,61.553704,2523.067529,2.885513,19.043018,76,49.502603
3,3,192.408932,72.094281,2020.812760,3.251630,19.473784,35,72.310220
4,4,188.675580,74.561067,2459.594194,1.789144,20.945017,49,67.864721


In [62]:
df.columns

Index(['Unnamed: 0', 'Height', 'Weight', 'Calories_Intake', 'Exercise_Time',
       'BMI', 'Age', 'Resting_Heart_Rate'],
      dtype='object')

In [63]:
df.drop('Unnamed: 0', axis = 1, inplace=True)

In [64]:
# Standardize the feature

scaler = StandardScaler()
dt_std = scaler.fit_transform(df)

# Perform PCA
pca = PCA()
df_pca = pca.fit_transform(df_std)



NameError: name 'df_std' is not defined

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(len(df.columns))])

NameError: name 'df_pca' is not defined

In [ ]:
# The transformed data is an array, convert it back into a dataframe
df_pca = pd.DataFrame(df_pca, columns=[f'PC{I+1}' FOR i in range(len(df.columns))])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1276206763.py, line 2)